In [15]:
import networkx as nx
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import json
import numpy as np

In [16]:
# Load the datasets from the assignment_1_data folder
path = "World_Trade_Web/"
files = [f for f in listdir(path) if isfile(join(path, f))]
gmls = [nx.read_graphml(path + f) for f in files]

graphs = {k: v for k, v in zip(files, gmls)}

In [9]:
# Test graph from the paper
# Do not execute for the assignment

G=nx.DiGraph()
G.add_nodes_from(["A","B","C","D","E","F","G","H"])
G.add_edges_from([("A","B", {"weight": 10}),("A","E", {"weight":3}), ("C", "A", {"weight":1}), ("D", "A",{"weight":2}), ("B", "G",{"weight":5}), ("B", "F",{"weight":4}), ("H", "B",{"weight":6})])

graphs={}
graphs["test"] = G

In [17]:
strengths = {}

for k, G in graphs.items():
    in_ = {}
    out = {}
    for source, target, weight in G.edges(data="weight"):
        if source not in out:
            out[source] = 0
        if source not in in_:
            in_[source] = 0
        if target not in out:
            out[target] = 0
        if target not in in_:
            in_[target] = 0
        if weight is not None:
            out[source] += weight
            in_[target] += weight
    strengths[k] = {"in": in_, "out": out}

In [18]:
W = {}
s_bar_sou_out = {}
s_bar_sou_in = {}
s_bar_tar_in = {}
s_bar_tar_out = {}
sigma_sou_out = {}
sigma_sou_in = {}
sigma_tar_in = {}
sigma_tar_out = {}

for k, strength in strengths.items():
    W[k] = sum(strength["out"].values())
    s_bar_sou_out[k] = 0
    s_bar_sou_in[k] = 0
    s_bar_tar_in[k] = 0
    s_bar_tar_out[k] = 0
    for node in strength["out"]:
        s_bar_sou_out[k] += strength["out"][node]*strength["out"][node]
        s_bar_sou_in[k] += strength["out"][node]*strength["in"][node]
        s_bar_tar_out[k] += strength["in"][node]*strength["out"][node]
        s_bar_tar_in[k] += strength["in"][node]*strength["in"][node]
    s_bar_sou_out[k] /= W[k]
    s_bar_sou_in[k] /= W[k]
    s_bar_tar_in[k] /= W[k]
    s_bar_tar_out[k] /= W[k]
    sigma_sou_out[k] = 0
    sigma_sou_in[k] = 0
    sigma_tar_in[k] = 0
    sigma_tar_out[k] = 0
    for node in strength["out"]:
        sigma_sou_out[k] += strength["out"][node]*(strength["out"][node] - s_bar_sou_out[k])**2
        sigma_sou_in[k] += strength["out"][node]*(strength["in"][node] - s_bar_sou_in[k])**2
        sigma_tar_in[k] += strength["in"][node]*(strength["in"][node] - s_bar_tar_in[k])**2
        sigma_tar_out[k] += strength["in"][node]*(strength["out"][node] - s_bar_tar_out[k])**2
    sigma_sou_out[k] = np.sqrt(sigma_sou_out[k]/W[k])
    sigma_sou_in[k] = np.sqrt(sigma_sou_in[k]/W[k])
    sigma_tar_in[k] = np.sqrt(sigma_tar_in[k]/W[k])
    sigma_tar_out[k] = np.sqrt(sigma_tar_out[k]/W[k])

In [19]:
rho_out_in = {}
rho_out_out = {}
rho_in_in = {}
rho_in_out = {}

for k, G in graphs.items():
    rho_out_in[k] = 0
    rho_out_out[k] = 0
    rho_in_in[k] = 0
    rho_in_out[k] = 0
    for source, target, weight in G.edges(data="weight"):
        if weight is not None:
            rho_out_in[k] += weight*(strengths[k]["out"][source] - s_bar_sou_out[k])*(strengths[k]["in"][target] - s_bar_tar_in[k])
            rho_out_out[k] += weight*(strengths[k]["out"][source] - s_bar_sou_out[k])*(strengths[k]["out"][target] - s_bar_tar_out[k])
            rho_in_in[k] += weight*(strengths[k]["in"][source] - s_bar_sou_in[k])*(strengths[k]["in"][target] - s_bar_tar_in[k])
            rho_in_out[k] += weight*(strengths[k]["in"][source] - s_bar_sou_in[k])*(strengths[k]["out"][target] - s_bar_tar_out[k])
    rho_out_in[k] /= W[k]*sigma_sou_out[k]*sigma_tar_in[k]
    rho_out_out[k] /= W[k]*sigma_sou_out[k]*sigma_tar_out[k]
    rho_in_in[k] /= W[k]*sigma_sou_in[k]*sigma_tar_in[k]
    rho_in_out[k] /= W[k]*sigma_sou_in[k]*sigma_tar_out[k]

    print(k + " | " + "rho out-in: " + str(rho_out_in[k]) + "| rho out-out: " + str(rho_out_out[k]) + "| rho in-in: " + str(rho_in_in[k]) + "| rho in-out: " + str(rho_in_out[k]))


WDN_1992.txt.graphml | rho out-in: -0.2664117215330637| rho out-out: -0.26076773855213525| rho in-in: -0.23043052971239772| rho in-out: -0.2178041687531879
WDN_1993.txt.graphml | rho out-in: -0.2332948861604653| rho out-out: -0.21690124032555627| rho in-in: -0.23436902905204038| rho in-out: -0.2021385793483107
WDN_1994.txt.graphml | rho out-in: -0.2280788632262324| rho out-out: -0.22662914473192905| rho in-in: -0.22585786853596368| rho in-out: -0.22015233241342597
WDN_1995.txt.graphml | rho out-in: -0.21154591029146982| rho out-out: -0.20812357271965518| rho in-in: -0.21449016601768575| rho in-out: -0.2074765749509863
WDN_1996.txt.graphml | rho out-in: -0.19698652263843464| rho out-out: -0.19471663098921604| rho in-in: -0.2022276276627051| rho in-out: -0.19651218763930592
WDN_1997.txt.graphml | rho out-in: -0.1918257224497967| rho out-out: -0.19315231407495667| rho in-in: -0.1932977484020739| rho in-out: -0.19017801772442888
WDN_1998.txt.graphml | rho out-in: -0.20399322491207095| rho 